# Pydantic — Jupyter Notebook (Beginner → Advanced)

## What is Pydantic? 
Pydantic uses Python type hints to validate and parse data into Python objects. It converts input (e.g., strings) into the annotated types when possible and raises errors when invalid. BaseModel to define schemas, Field() for metadata/constraints, and validators for custom rules.

### Set Up

In [1]:
# In a notebook cell (use ! to run shell commands)
# !pip install -U "pydantic>=2.0"
# !pip install python-dotenv  # optional, for settings exercises

# Import the essentials
from pydantic import BaseModel, Field, ValidationError, TypeAdapter
from pydantic import field_validator, model_validator
import datetime
from typing import List, Optional, Union, Literal, Dict

### Basic BaseModel usage — beginner

In [ ]:
class User(BaseModel):
    id: int
    name: str 
    signup_ts: Optional[datetime.datetime] = None
    friends: List[int] = []

# Example usage
raw = {"id": "123", "name": "Alice", "signup_ts": "2023-07-01T12:00:00", "friends": [1, "2"]}
user = User.model_validate(raw)
user

User(id=123, name='Alice', signup_ts=datetime.datetime(2023, 7, 1, 12, 0), friends=[1, 2])

In [3]:
raw = {"id": "abc", "name": "Alice", "signup_ts": "2023-07-01T12:00:00", "friends": [1, "2"]}
user = User.model_validate(raw)

ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

#### without Pydantic, the data type validation

In [9]:
#Lets validate by using user defined functins

class Person():
    def __init__(self,name,age,city):
        if not isinstance(name,str):
            raise TypeError ("Name must be integer")
        if not isinstance(age, int):
            raise TypeError("Age must be number")
        if not isinstance(city, str):
            raise TypeError("Age must be number")
        self.name = name
        self.age = age
        self.city = city
    def __str__(self):
        return f"Person(name={self.name}, age={self.age}, city={self.city})"   

In [10]:
# Valid Input
p=Person("Chandra", 30,"Nuremberg")    
print(p)

Person(name=Chandra, age=30, city=Nuremberg)


In [11]:
#invalid input
p = Person("Chandra", "30","Nuremberg")    
print(p)
#It just raised the exception not much details

TypeError: Age must be number

### Fields, defaults, and Field()

Field() adds metadata and validation constraints like how we did custom functions

In [6]:
class Product(BaseModel):
    name: str = Field(..., min_length=1)
    price: float = Field(..., gt=0)
    tags: List[str] = Field(default_factory=list)

# valid
p = Product.model_validate({"name": "Banana", "price": "0.5"})
p


Product(name='Banana', price=0.5, tags=[])

In [7]:
# invalid (price <= 0) -> raises
try:
    Product.model_validate({"name": "Bad", "price": -2})
except ValidationError as ve:
    print(ve)

1 validation error for Product
price
  Input should be greater than 0 [type=greater_than, input_value=-2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than


### Custom validators — field and model validators

Use @field_validator for single-field logic and @model_validator for cross-field checks.

In [12]:
class Order(BaseModel):
    item: str
    quantity: int
    discount: Optional[float] = None

    @field_validator('quantity')
    def check_quantity(cls, v):
        if v <= 0:
            raise ValueError('quantity must be > 0')
        return v

    @model_validator(mode='after')
    def check_discount(cls, model):
        if model.discount is not None and not (0 <= model.discount <= 1):
            raise ValueError('discount must be between 0 and 1')
        return model

# Test
Order.model_validate({"item": "apple", "quantity": 3, "discount": 0.1})


Order(item='apple', quantity=3, discount=0.1)

In [13]:
# Test
Order.model_validate({"item": "apple", "quantity": 3, "discount": 2})

ValidationError: 1 validation error for Order
  Value error, discount must be between 0 and 1 [type=value_error, input_value={'item': 'apple', 'quantity': 3, 'discount': 2}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

### TypeAdapter (validate standalone types) and Built in Validators

TypeAdapter enables parsing/validation for arbitrary type hints without a model.

In [14]:
from pydantic import TypeAdapter
adapter = TypeAdapter(List[int])
adapter.validate_python(["1", 2, "3"])  # -> [1, 2, 3]

[1, 2, 3]

In [21]:
#Built-in types and constrained types
from pydantic import EmailStr
class Contact(BaseModel):
    email: EmailStr

Contact.model_validate({"email": "not-an-email"})  # raises

ValidationError: 1 validation error for Contact
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]

### Model with Nested Models
Create complex structures with nested models and Default, constraights etc

In [17]:
class Address(BaseModel):
    dr_num:int
    strt: str
    zip: int

In [18]:
class Customer(BaseModel):
    cust_id:int=Field(le=9999,gt=0)
    name: str
    email_id:str=Field(default_factory=lambda: "user@domain.com", description='mandatory email address')
    mobile: Optional[int]=None
    telephne: int=Field(default=None)
    address: Address #<- Nested class

In [20]:
cust=Customer(cust_id=1,name='chandra',email_id='chandra@gmail.com',address={"dr_num":12,"strt":'Schwabstr',"zip":90763})
cust

Customer(cust_id=1, name='chandra', email_id='chandra@gmail.com', mobile=None, telephne=None, address=Address(dr_num=12, strt='Schwabstr', zip=90763))